<a href="https://colab.research.google.com/github/armandossrecife/piloto/blob/main/analise_sas2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Install Dependencies

* [GitPython](https://gitpython.readthedocs.io/en/stable) - to clone git repositories
* [CLOC](https://github.com/AlDanial/cloc) - to calculate CLOC and other source code metrics





In [1]:
!pip3 install gitpython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.0 MB/s eta 0:00:00


In [2]:
!sudo apt install cloc

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libalgorithm-diff-perl libalgorithm-diff-xs-perl libb-hooks-endofscope-perl
  libb-hooks-op-check-perl libclass-method-modifiers-perl
  libclass-xsaccessor-perl libdevel-callchecker-perl
  libdynaloader-functions-perl libimport-into-perl
  libmodule-implementation-perl libmodule-runtime-perl libmoo-perl
  libnamespace-clean-perl libpackage-stash-perl libpackage-stash-xs-perl
  libparallel-forkmanager-perl libparams-classify-perl libregexp-common-perl
  librole-tiny-perl libsub-exporter-progressive-perl libsub-identify-perl
  libsub-name-perl libsub-quote-perl libtry-tiny-perl libvariable-magic-perl
Suggested packages:
  libscalar-number-perl
The following NEW packages will be installed:
  cloc libalgorithm-diff-perl libalgorithm-diff-xs-perl
  libb-hooks-endofscope-perl libb-hooks-op-check-perl
  libclass-method-modifiers-perl libclass-

# 2. Dataset do Darius Sas

On the relation between architectural smells and source code changes - https://onlinelibrary.wiley.com/doi/full/10.1002/smr.2398

replication kit - https://zenodo.org/record/4459015

In [3]:
!wget https://raw.githubusercontent.com/armandossrecife/teste/main/my_repositories.csv

--2023-08-19 16:15:08--  https://raw.githubusercontent.com/armandossrecife/teste/main/my_repositories.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1290 (1.3K) [text/plain]
Saving to: ‘my_repositories.csv’

my_repositories.csv 100%[===================>]   1.26K  --.-KB/s    in 0s      

2023-08-19 16:15:08 (48.1 MB/s) - ‘my_repositories.csv’ saved [1290/1290]



## 2.1 Replication Kit data and scripts

In [4]:
!wget https://zenodo.org/record/4459015/files/replication-package.zip

--2023-08-19 16:15:14--  https://zenodo.org/record/4459015/files/replication-package.zip
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 95506210 (91M) [application/octet-stream]
Saving to: ‘replication-package.zip’

replication-package 100%[===================>]  91.08M   106MB/s    in 0.9s    

2023-08-19 16:15:16 (106 MB/s) - ‘replication-package.zip’ saved [95506210/95506210]



In [5]:
!unzip replication-package.zip

Archive:  replication-package.zip
   creating: replication-package/
   creating: replication-package/csv-2019-12-09/
  inflating: replication-package/csv-2019-12-09/projects.csv  
  inflating: replication-package/csv-2019-12-09/components.csv  
  inflating: replication-package/csv-2019-12-09/affected.csv  
  inflating: replication-package/csv-2019-12-09/smells.csv  
  inflating: replication-package/protocol.pdf  
   creating: replication-package/plots/
  inflating: replication-package/plots/aux-loc-boxplot-individual-project.png  
  inflating: replication-package/plots/aux-avrg-change-by-age-all.png  
  inflating: replication-package/plots/aux-loc-density-individual-project.png  
  inflating: replication-package/plots/aux-nsmell-locgroup-individual-project.png  
  inflating: replication-package/plots/aux-smell-count-individual-project.png  
  inflating: replication-package/plots/rq1b-changeFreq-nsmells-loess.png  
  inflating: replication-package/plots/rq1b-changeSize-nsmells-loess.png

In [6]:
import pandas as pd

df_smells = pd.read_csv('replication-package/csv-2019-12-09/smells.csv')
df_projects = pd.read_csv('replication-package/csv-2019-12-09/projects.csv')
df_components = pd.read_csv('replication-package/csv-2019-12-09/components.csv')
df_affected = pd.read_csv('replication-package/csv-2019-12-09/affected.csv')

<ipython-input-6-3284ae094b4a>:5: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_components = pd.read_csv('replication-package/csv-2019-12-09/components.csv')


## 2.2 Architectural Smells

1. Dataset with architectural smells

In [7]:
df_smells.head(3)

,project,age,firstAppeared,lastDetected,smellType,uniqueSmellID,version,versionIndex,versionDate.x,smellIdInVersion,...,shape,size,strength,affectedElements,firstAppearedIndex,lastDetectedIndex,versionDate.y,nPackages,nClasses,pageRankWeighted
0,accumulo,100,1-4558b9a25d11c5ac8a03c3c671472786a49e8e98,101-2e8144ec89e74d37c7ff97d301051fea747e05c7,unstableDep,1,5-917399157ace8ae691164d44db8eec3953b205f7,5,19-2-2012,135496,...,NaN,4,0.05,"[org.apache.accumulo.core.bloomfilter, org.apa...",NaN,NaN,19-2-2012,97,1937,0.084671
1,accumulo,100,1-4558b9a25d11c5ac8a03c3c671472786a49e8e98,101-2e8144ec89e74d37c7ff97d301051fea747e05c7,unstableDep,1,77-646b059eb0b2de3bffe329d744c47bd9890103c7,77,17-11-2017,1024721,...,NaN,5,0.00,"[org.apache.accumulo.core.bloomfilter, org.apa...",NaN,NaN,17-11-2017,74,2812,0.039795
2,accumulo,100,1-4558b9a25d11c5ac8a03c3c671472786a49e8e98,101-2e8144ec89e74d37c7ff97d301051fea747e05c7,unstableDep,1,94-9aae33bbf39098c8aa601c040754cf36bfbdf1d4,94,9-4-2019,1508745,...,NaN,5,0.00,"[org.apache.accumulo.core.bloomfilter, org.apa...",NaN,NaN,9-4-2019,76,2833,0.045560


In [8]:
df_smells.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271739 entries, 0 to 271738
Data columns (total 40 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   project                  271739 non-null  object 
 1   age                      271739 non-null  int64  
 2   firstAppeared            271739 non-null  object 
 3   lastDetected             271739 non-null  object 
 4   smellType                271739 non-null  object 
 5   uniqueSmellID            271739 non-null  int64  
 6   version                  271739 non-null  object 
 7   versionIndex             271739 non-null  int64  
 8   versionDate.x            271739 non-null  object 
 9   smellIdInVersion         271739 non-null  int64  
 10  affectedClassesRatio     8674 non-null    float64
 11  affectedComponentType    271739 non-null  object 
 12  affectedDesignLevel      207182 non-null  object 
 13  afferentAffectedRatio    8674 non-null    float64
 14  avrg

2. Projects (repositories) with identified Architectural Smells

In [9]:
df_smells['project'].unique()

array(['accumulo', 'activemq', 'ant-ivy', 'cassandra', 'chukwa', 'druid',
       'guava', 'httpcomponents-client', 'jackrabbit', 'jackson-databind',
       'jena', 'jspwiki', 'lucene-solr', 'mina', 'pdfbox', 'pgjdbc',
       'poi', 'struts', 'testng', 'tika', 'xerces2-j'], dtype=object)

In [10]:
qty_complete_repositories_as = len(df_smells['project'].unique())
print(f"Amount of repositories with complete (AS) data {qty_complete_repositories_as}")

Amount of repositories with complete (AS) data 21


In [11]:
for each in df_smells['project'].unique():
  print(each)

accumulo
activemq
ant-ivy
cassandra
chukwa
druid
guava
httpcomponents-client
jackrabbit
jackson-databind
jena
jspwiki
lucene-solr
mina
pdfbox
pgjdbc
poi
struts
testng
tika
xerces2-j


3. Types of Architectural Smells identified

In [12]:
df_smells.smellType.unique()

array(['unstableDep', 'cyclicDep', 'hubLikeDep', 'godComponent'],
      dtype=object)

In [13]:
for each in df_smells.smellType.unique():
  print(each)

unstableDep
cyclicDep
hubLikeDep
godComponent


4. Types of Components Affected by Architectural Smells

In [14]:
df_smells.affectedComponentType.unique()

array(['package', 'class'], dtype=object)

In [15]:
for each in df_smells.affectedComponentType.unique():
  print(each)

package
class


5. Classes affected by Architectural Smells

In [16]:
df_smells_classes_affected = df_smells.query("affectedComponentType=='class'")
df_smells_classes_affected[['project', 'smellType', 'affectedComponentType', 'affectedElements']]

,project,smellType,affectedComponentType,affectedElements
124,accumulo,cyclicDep,class,"[parser.TreeNode, parser.TreeNode$PostorderEnu..."
327,accumulo,cyclicDep,class,[org.apache.accumulo.core.security.tokens.Kerb...
678,accumulo,cyclicDep,class,[org.apache.accumulo.core.security.tokens.User...
1359,accumulo,cyclicDep,class,[org.apache.accumulo.core.client.summary.Summa...
1360,accumulo,cyclicDep,class,[org.apache.accumulo.core.client.summary.Summa...
...,...,...,...,...
271712,xerces2-j,cyclicDep,class,"[org.apache.xerces.dom.AttrImpl, org.apache.xe..."
271713,xerces2-j,cyclicDep,class,"[org.apache.xerces.dom.AttrImpl, org.apache.xe..."
271714,xerces2-j,cyclicDep,class,"[org.apache.xerces.dom.AttrImpl, org.apache.xe..."
271715,xerces2-j,cyclicDep,class,"[org.apache.xerces.dom.AttrImpl, org.apache.xe..."


### Cassandra classes affected by Archictural Smells

In [17]:
df_cassandra_smells_classes_affected = df_smells_classes_affected.query("project=='cassandra'")
df_cassandra_smells_classes_affected[['affectedElements']]

,affectedElements
50351,"[org.apache.cassandra.db.ColumnFamily, org.apa..."
50352,"[org.apache.cassandra.db.ColumnFamily, org.apa..."
50359,"[org.apache.cassandra.db.Keyspace, org.apache...."
50360,"[org.apache.cassandra.db.Keyspace, org.apache...."
50361,"[org.apache.cassandra.db.Keyspace, org.apache...."
...,...
91854,"[org.apache.cassandra.config.CFMetaData, org.a..."
91855,"[org.apache.cassandra.config.CFMetaData, org.a..."
91893,"[org.apache.cassandra.db.Mutation, org.apache...."
91894,"[org.apache.cassandra.db.Mutation, org.apache...."


### ActiveMQ Classes Affected by Architectural Smells

In [18]:
df_activemq_smells_classes_affected = df_smells_classes_affected.query("project=='activemq'")
df_activemq_smells_classes_affected[['affectedElements']]

,affectedElements
8088,"[org.apache.activemq.ActiveMQConnection, org.a..."
8089,"[org.apache.activemq.ActiveMQConnection, org.a..."
8090,"[org.apache.activemq.ActiveMQConnection, org.a..."
8091,"[org.apache.activemq.ActiveMQConnection, org.a..."
8092,"[org.apache.activemq.ActiveMQConnection, org.a..."
...,...
33702,"[org.apache.activemq.ActiveMQConnection, org.a..."
33734,"[org.apache.activemq.broker.BrokerService, org..."
33735,"[org.apache.activemq.broker.BrokerService, org..."
33736,"[org.apache.activemq.broker.BrokerService, org..."


### Guava classes affected by Architectural Smells

In [19]:
#guava
df_guava_smells_classes_affected = df_smells_classes_affected.query("project=='guava'")
df_guava_smells_classes_affected[['affectedElements']]

,affectedElements
100576,"[com.google.common.base.CharMatcher, com.googl..."
100577,"[com.google.common.base.CharMatcher, com.googl..."
100578,"[com.google.common.base.CharMatcher, com.googl..."
100579,"[com.google.common.base.CharMatcher, com.googl..."
100580,"[com.google.common.base.CharMatcher, com.googl..."
...,...
105088,"[com.google.common.collect.ImmutableTable, com..."
105089,"[com.google.common.collect.ImmutableTable, com..."
105090,"[com.google.common.collect.ImmutableTable, com..."
105091,"[com.google.common.collect.ImmutableTable, com..."


## 2.3 Projects (repositories)

In [20]:
df_projects.head(3)

,project,version,versionDate,versionIndex,nPackages,nClasses
0,accumulo,1-4558b9a25d11c5ac8a03c3c671472786a49e8e98,28-10-2011,1,85,1596
1,accumulo,3-968ad7fe2efbdcf9bfcd77be1b8b47d340aee99e,23-12-2011,3,109,2031
2,accumulo,4-6d8d0acc548a79ce1e6fbe15dc9c0dabd646c6ca,20-1-2012,4,96,1931


In [21]:
df_projects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3087 entries, 0 to 3086
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   project       3087 non-null   object
 1   version       3087 non-null   object
 2   versionDate   3087 non-null   object
 3   versionIndex  3087 non-null   int64 
 4   nPackages     3087 non-null   int64 
 5   nClasses      3087 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 144.8+ KB


## 2.4 Components

In [22]:
df_components.head()

,project,name,type,version,versionIndex,numOfClassesInPackage,linesOfCode,freqOfChanges,percCommitsClassChanged,changeHasOccurredMetric,percCommitsPackChanged,totalAmountOfChanges
0,accumulo,org.apache.accumulo.core.master.thrift.MasterC...,class,1-4558b9a25d11c5ac8a03c3c671472786a49e8e98,1,0,47,0,0.0,0,0.0,0
1,accumulo,org.apache.accumulo.core.file.BloomFilterLayer...,class,1-4558b9a25d11c5ac8a03c3c671472786a49e8e98,1,0,94,0,0.0,0,0.0,0
2,accumulo,org.apache.accumulo.core.client.impl.TabletLoc...,class,1-4558b9a25d11c5ac8a03c3c671472786a49e8e98,1,0,36,0,0.0,0,0.0,0
3,accumulo,org.apache.accumulo.server.constraints.Metadat...,class,1-4558b9a25d11c5ac8a03c3c671472786a49e8e98,1,0,196,0,0.0,0,0.0,0
4,accumulo,org.apache.accumulo.core.tabletserver.thrift.M...,class,1-4558b9a25d11c5ac8a03c3c671472786a49e8e98,1,0,41,0,0.0,0,0.0,0


In [23]:
df_components.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3881915 entries, 0 to 3881914
Data columns (total 12 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   project                  object 
 1   name                     object 
 2   type                     object 
 3   version                  object 
 4   versionIndex             int64  
 5   numOfClassesInPackage    int64  
 6   linesOfCode              int64  
 7   freqOfChanges            int64  
 8   percCommitsClassChanged  float64
 9   changeHasOccurredMetric  object 
 10  percCommitsPackChanged   float64
 11  totalAmountOfChanges     int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 355.4+ MB


In [24]:
df_components.project.unique()

array(['accumulo', 'activemq', 'ant-ivy', 'cassandra', 'chukwa', 'druid',
       'guava', 'httpcomponents-client', 'jackrabbit', 'jackson-databind',
       'jena', 'jspwiki', 'lucene-solr', 'mina', 'pdfbox', 'pgjdbc',
       'poi', 'struts', 'testng', 'tika', 'xerces2-j'], dtype=object)

### Cassandra Components

In [25]:
df_cassandra_classes = df_components.query("type=='class' and project=='cassandra'")
df_cassandra_classes

,project,name,type,version,versionIndex,numOfClassesInPackage,linesOfCode,freqOfChanges,percCommitsClassChanged,changeHasOccurredMetric,percCommitsPackChanged,totalAmountOfChanges
732432,cassandra,org.apache.cassandra.service.Cassandra$getStri...,class,1-cf56f8f3acbeac0ddff9ee3131ad7abe2a919ba3,1,0,105,0,0.000000,0,0.0,0
732434,cassandra,org.apache.cassandra.locator.RackAwareStrategy,class,1-cf56f8f3acbeac0ddff9ee3131ad7abe2a919ba3,1,0,127,0,0.000000,0,0.0,0
732435,cassandra,org.apache.cassandra.io.FastBufferedInputStream,class,1-cf56f8f3acbeac0ddff9ee3131ad7abe2a919ba3,1,0,163,0,0.000000,0,0.0,0
732436,cassandra,org.apache.cassandra.db.ColumnFactory,class,1-cf56f8f3acbeac0ddff9ee3131ad7abe2a919ba3,1,0,13,0,0.000000,0,0.0,0
732437,cassandra,org.apache.cassandra.io.SSTable,class,1-cf56f8f3acbeac0ddff9ee3131ad7abe2a919ba3,1,0,598,0,0.000000,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1018985,cassandra,org.apache.cassandra.tools.nodetool.RefreshSiz...,class,137-5d6b1c7efb702d5182d226f049005ee1678eee42,137,0,6,2,1.459854,false,0.0,0
1018986,cassandra,org.apache.cassandra.metrics.ClientMetrics,class,137-5d6b1c7efb702d5182d226f049005ee1678eee42,137,0,16,7,5.109489,false,0.0,0
1018987,cassandra,org.apache.cassandra.metrics.CompactionMetrics$3,class,137-5d6b1c7efb702d5182d226f049005ee1678eee42,137,0,8,6,4.379562,false,0.0,0
1018988,cassandra,org.apache.cassandra.utils.MerkleTree$OffHeapI...,class,137-5d6b1c7efb702d5182d226f049005ee1678eee42,137,0,62,1,0.729927,false,0.0,0


### ActiveMQ Components

In [26]:
df_activemq_classes = df_components.query("type=='class' and project=='activemq'")
df_activemq_classes

,project,name,type,version,versionIndex,numOfClassesInPackage,linesOfCode,freqOfChanges,percCommitsClassChanged,changeHasOccurredMetric,percCommitsPackChanged,totalAmountOfChanges
244936,activemq,org.activemq.sampler.Producer,class,1-80d129918854d1409869683d284797233c10d9cd,1,0,326,0,0.000000,0,0.0,0
244937,activemq,org.activemq.openwire.v1.ActiveMQTempQueueMars...,class,1-80d129918854d1409869683d284797233c10d9cd,1,0,17,0,0.000000,0,0.0,0
244938,activemq,org.activemq.util.Callback,class,1-80d129918854d1409869683d284797233c10d9cd,1,0,2,0,0.000000,0,0.0,0
244939,activemq,org.apache.activeio.xnet.ServiceDaemon,class,1-80d129918854d1409869683d284797233c10d9cd,1,0,125,0,0.000000,0,0.0,0
244940,activemq,org.activemq.transport.stomp.Disconnect,class,1-80d129918854d1409869683d284797233c10d9cd,1,0,6,0,0.000000,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
653590,activemq,org.apache.activemq.openwire.v5.ExceptionRespo...,class,175-0325fd2eff3c2fd37c422a8450770e91820613c3,175,0,33,3,1.851852,false,0.0,0
653591,activemq,org.apache.activemq.xbean.BrokerFactoryBean$1,class,175-0325fd2eff3c2fd37c422a8450770e91820613c3,175,0,4,4,2.469136,false,0.0,0
653592,activemq,org.apache.activemq.openwire.v2.ReplayCommandM...,class,175-0325fd2eff3c2fd37c422a8450770e91820613c3,175,0,36,6,3.703704,false,0.0,0
653593,activemq,org.apache.activemq.openwire.v5.DiscoveryEvent...,class,175-0325fd2eff3c2fd37c422a8450770e91820613c3,175,0,38,3,1.851852,false,0.0,0


### Guava Components

In [27]:
df_guava_classes = df_components.query("type=='class' and project=='guava'")
df_guava_classes

,project,name,type,version,versionIndex,numOfClassesInPackage,linesOfCode,freqOfChanges,percCommitsClassChanged,changeHasOccurredMetric,percCommitsPackChanged,totalAmountOfChanges
1157019,guava,com.google.common.util.concurrent.Futures,class,2-a194b64f31a7453a9be7e4573b2d92afab3aa37e,2,0,229,0,0.000000,0,0.0,0
1157020,guava,com.google.common.io.OutputSupplier,class,2-a194b64f31a7453a9be7e4573b2d92afab3aa37e,2,0,2,0,0.000000,0,0.0,0
1157021,guava,com.google.common.base.Splitter$2$1,class,2-a194b64f31a7453a9be7e4573b2d92afab3aa37e,2,0,18,0,0.000000,0,0.0,0
1157022,guava,com.google.common.base.CharMatcher$13,class,2-a194b64f31a7453a9be7e4573b2d92afab3aa37e,2,0,14,0,0.000000,0,0.0,0
1157023,guava,com.google.common.base.Defaults,class,2-a194b64f31a7453a9be7e4573b2d92afab3aa37e,2,0,21,0,0.000000,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1275567,guava,com.google.common.primitives.Booleans,class,116-82988bb2cafb95fd5dde57720024112c20891ac3,116,0,285,3,3.488372,false,0.0,0
1275568,guava,com.google.common.util.concurrent.Striped$5,class,116-82988bb2cafb95fd5dde57720024112c20891ac3,116,0,5,5,5.813953,false,0.0,0
1275569,guava,com.google.common.hash.PrimitiveSink,class,116-82988bb2cafb95fd5dde57720024112c20891ac3,116,0,16,3,3.488372,false,0.0,0
1275570,guava,com.google.common.util.concurrent.AbstractServ...,class,116-82988bb2cafb95fd5dde57720024112c20891ac3,116,0,9,5,5.813953,false,0.0,0


## 2.5 Affected

In [28]:
df_affected.head()

,project,name,type,version,affectedByUniqueSmellId
0,accumulo,org.apache.accumulo.core.file,package,40-58af0ce9f04cb1efa59f69c8abdeb87e3c73504c,7
1,accumulo,org.apache.accumulo.core.file,package,38-59875c298617221282825fb870dbe7abb86ea4c4,1
2,accumulo,org.apache.accumulo.core.file,package,8-1e1cc2484e61e1de17ecabc6d7fb46131eece2e8,7
3,accumulo,org.apache.accumulo.core.file,package,74-82a15686893cc2f1b26777664093fa1ac312248a,1113
4,accumulo,org.apache.accumulo.core.file,package,97-436f404e247114737eb6a499799cf90f1a201620,1534


In [29]:
df_affected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1836160 entries, 0 to 1836159
Data columns (total 5 columns):
 #   Column                   Dtype 
---  ------                   ----- 
 0   project                  object
 1   name                     object
 2   type                     object
 3   version                  object
 4   affectedByUniqueSmellId  int64 
dtypes: int64(1), object(4)
memory usage: 70.0+ MB


In [30]:
df_affected.project.unique()

array(['accumulo', 'activemq', 'ant-ivy', 'cassandra', 'chukwa', 'druid',
       'guava', 'httpcomponents-client', 'jackrabbit', 'jackson-databind',
       'jena', 'jspwiki', 'lucene-solr', 'mina', 'pdfbox', 'pgjdbc',
       'poi', 'struts', 'testng', 'tika', 'xerces2-j'], dtype=object)

### Cassandra components affected by AS

In [31]:
df_cassandra_classes_affected_by_as = df_affected.query("type=='class' and project=='cassandra'")
df_cassandra_classes_affected_by_as

,project,name,type,version,affectedByUniqueSmellId
330300,cassandra,org.apache.cassandra.db.CommitLog,class,7-64c38a2657ae363b22dcbf9bf3ea0b518d75681d,55
330301,cassandra,org.apache.cassandra.db.CommitLog,class,2-50c1987265f8ddaf5032a7ace07b2462aba4f09b,59
330302,cassandra,org.apache.cassandra.db.CommitLog,class,2-50c1987265f8ddaf5032a7ace07b2462aba4f09b,54
330303,cassandra,org.apache.cassandra.db.CommitLog,class,8-92098fe695dc054d9c6706ae873d68d72947d01f,107
330304,cassandra,org.apache.cassandra.db.CommitLog,class,19-27f6691ca35f52ea8b0853dd1bf18b67e89f5fc9,327
...,...,...,...,...,...
793367,cassandra,org.apache.cassandra.db.SSTableImporter$Options,class,132-6b9575af7e244efe98943e5e7f92b33e252218a3,6343
793368,cassandra,org.apache.cassandra.db.SSTableImporter$Options,class,125-801cb70ee811c956e987718a00695638d5bec1b6,17053
793369,cassandra,org.apache.cassandra.db.SSTableImporter$Options,class,121-13150b001a8ddf82a77ac9525c446b7e9e32325c,17053
793370,cassandra,org.apache.cassandra.db.SSTableImporter$Options,class,131-3a87604e4c83939f05d1a51e577e69d45dc7345d,17053


### ActiveMQ components affected by AS

In [32]:
df_activemq_classes_affected_by_as = df_affected.query("type=='class' and project=='activemq'")
df_activemq_classes_affected_by_as

,project,name,type,version,affectedByUniqueSmellId
39480,activemq,org.activemq.ActiveMQConnection,class,1-80d129918854d1409869683d284797233c10d9cd,30
39481,activemq,org.activemq.ActiveMQConnection,class,1-80d129918854d1409869683d284797233c10d9cd,49
39482,activemq,org.activemq.ActiveMQConnection,class,1-80d129918854d1409869683d284797233c10d9cd,31
39483,activemq,org.activemq.ActiveMQConnection,class,1-80d129918854d1409869683d284797233c10d9cd,47
39484,activemq,org.activemq.ActiveMQConnection,class,1-80d129918854d1409869683d284797233c10d9cd,50
...,...,...,...,...,...
204046,activemq,org.apache.activemq.transport.amqp.protocol.Am...,class,175-0325fd2eff3c2fd37c422a8450770e91820613c3,5661
204047,activemq,org.apache.activemq.transport.amqp.protocol.Am...,class,162-cee7f014fa4afca04681066946e5a2b6811141af,5661
204048,activemq,org.apache.activemq.transport.amqp.protocol.Am...,class,145-88efa01e119197fb043a890c54ddb37abce21ff4,5661
204049,activemq,org.apache.activemq.transport.amqp.protocol.Am...,class,126-5d353e241b0ba76ad1a0b42ef5c7a2ae54106860,4794


### Guava components affected by AS

In [33]:
df_guava_classes_affected_by_as = df_affected.query("type=='class' and project=='guava'")
df_guava_classes_affected_by_as

,project,name,type,version,affectedByUniqueSmellId
830686,guava,com.google.common.base.CharMatcher$Or,class,6-970ad12d61e18d5171f3a5e8e3d07c83f587d2ed,1
830687,guava,com.google.common.base.CharMatcher$Or,class,33-c523556ab7d0f05afadebd20e7768d4c16af8771,64
830688,guava,com.google.common.base.CharMatcher$Or,class,82-cb3a29fb67936c2bc52b1cfee08cedab62950282,146
830689,guava,com.google.common.base.CharMatcher$Or,class,9-803f44ac9c6a3457388394e952132fe18449cb9f,1
830690,guava,com.google.common.base.CharMatcher$Or,class,10-7d4849b19bb33ab4ad6c442a501f019b7dc00d89,1
...,...,...,...,...,...
843134,guava,com.google.common.io.BaseEncoding,class,113-23e4ad3ee650aa2954a7c9c5e60410de9cb342c9,99
843135,guava,com.google.common.io.BaseEncoding,class,75-b281c1fd5a960abf179fe8f4f280daca47ff2156,99
843136,guava,com.google.common.io.BaseEncoding,class,75-b281c1fd5a960abf179fe8f4f280daca47ff2156,135
843137,guava,com.google.common.io.BaseEncoding,class,72-82810968135e9ac4c9120f44b0936d478492066e,99


# 3. Details of the analyzed projects

Selection criteria:

* Project lifetime (in years)
* number of commits
* Number of released releases
* Number of files
* Number of .java files
* Number of participants (contributors)
* Relevance in the open-source community
** Amount of stars

In [34]:
repository_project_list = df_affected.project.unique().tolist()
repository_project_list

['accumulo',
 'activemq',
 'ant-ivy',
 'cassandra',
 'chukwa',
 'druid',
 'guava',
 'httpcomponents-client',
 'jackrabbit',
 'jackson-databind',
 'jena',
 'jspwiki',
 'lucene-solr',
 'mina',
 'pdfbox',
 'pgjdbc',
 'poi',
 'struts',
 'testng',
 'tika',
 'xerces2-j']

### Support classes to CLOC

In [35]:
import os
import subprocess
import json
import pandas as pd
import requests
import threading
import datetime
from git import Repo
import csv

class DirectoryAnalyzer:
    def __init__(self, target_directory, cloc_output_file="cloc_output.json"):
        self.target_directory = target_directory
        self.cloc_output_file = cloc_output_file

    def cloc(self):
        os.chdir(self.target_directory)
        try:
            # Run CLOC to analyze the repository and save the output as JSON
            subprocess.run(["cloc", "--json","."], stdout=open(self.cloc_output_file, "w"))
        except Exception as ex:
            print(f'Error: {str(ex)}')


        # Load JSON output from CLOC
        with open(self.cloc_output_file, "r") as json_file:
            cloc_data = json.load(json_file)

        # Extract desired metrics for Java files
        java_metrics = cloc_data.get("Java", {})
        self.total_unique_files = java_metrics.get("nFiles", 0)
        self.total_blank_loc = java_metrics.get("blank", 0)
        self.total_comment_loc = java_metrics.get("comment", 0)
        self.total_loc = java_metrics.get("code", 0)

        sum_metrics = cloc_data.get("SUM", {})
        self.total_nFiles = sum_metrics.get("nFiles",0)
        self.total_LOC_files = sum_metrics.get("code",0)

        return self.total_nFiles, self.total_unique_files, self.total_blank_loc, self.total_comment_loc, self.total_loc, self.total_LOC_files

class RepositoryAnalyzer:
    def __init__(self, target_directory):
        self.target_directory = target_directory

    def get_first_commit_date(self):
        command = ["git", "log", "--reverse", "--format=%ad", "--date=short"]
        result = subprocess.run(command, cwd=self.target_directory, stdout=subprocess.PIPE, text=True)
        self.first_commit_date = result.stdout.splitlines()[0]
        return self.first_commit_date

    def get_last_commit_date(self):
        command = ["git", "log", "-1", "--format=%ad", "--date=short"]
        result = subprocess.run(command, cwd=self.target_directory, stdout=subprocess.PIPE, text=True)
        self.last_commit_date = result.stdout.strip()
        return self.last_commit_date

    def get_total_commit_count(self):
        command = ["git", "rev-list", "--count", "HEAD"]
        result = subprocess.run(command, cwd=self.target_directory, stdout=subprocess.PIPE, text=True)
        self.total_commit_count = int(result.stdout.strip())
        return self.total_commit_count

    def get_total_release_count(self):
        git_tag_command = ["git", "tag"]
        wc_command = ["wc", "-l"]

        git_tag_result = subprocess.run(git_tag_command, cwd=self.target_directory, stdout=subprocess.PIPE, text=True)
        wc_result = subprocess.run(wc_command, input=git_tag_result.stdout, stdout=subprocess.PIPE, text=True)

        self.total_release_count = int(wc_result.stdout.strip())
        return self.total_release_count

def get_repository_name(url):
  repository_name = os.path.splitext(os.path.basename(url))[0]
  return repository_name

def clone_repository(url):
  target_directory = get_repository_name(url)
  subprocess.run(["git", "clone", url, target_directory])

def gera_my_dict(url, total_nFiles, total_unique_files, total_blank_loc, total_comment_loc, total_loc, total_LOC_files):
    name = get_repository_name(url)
    repository_path = '/content/'+name
    git_repo_analyzer = RepositoryAnalyzer(target_directory=repository_path)
    first_commit_date = git_repo_analyzer.get_first_commit_date()
    last_commit_date = git_repo_analyzer.get_last_commit_date()
    total_commit_count = git_repo_analyzer.get_total_commit_count()
    total_release_count = git_repo_analyzer.get_total_release_count()

    my_dict = {
        'name': [name],
        'files': [total_nFiles],
        'java': [total_unique_files],
        'blank_loc_java': [total_blank_loc],
        'comment_loc_java': [total_comment_loc],
        'loc_java': [total_loc],
        'loc_files': [total_LOC_files],
        'first_commit_date':[first_commit_date],
        'last_commit_date':[last_commit_date],
        'total_commit_count':[total_commit_count],
        'total_release_count':[total_release_count]
    }
    return my_dict

def gera_csv(url, my_dict):
  name = get_repository_name(url)
  my_df = pd.DataFrame.from_dict(my_dict)
  name = name + '.csv'
  name = '/content/'+name
  my_df.to_csv(name, index=False)
  return my_df

class RepoCloner:
    def __init__(self, qtd_repositorios, lista_urls):
        self.qtd_repositorios = qtd_repositorios
        self.lista_urls = lista_urls
        self.lista_threads = []
        self.lista_tempo_inicio = []
        self.lista_tempo_conclusao = []

    def my_clone(self, git_url, repo_dir):
        try:
            Repo.clone_from(git_url, repo_dir)
            tempo = datetime.datetime.now()
            print(f'Clonagem do repo {git_url} concluída com sucesso! -> instante: {tempo}')
            elemento = (repo_dir, tempo)
            self.lista_tempo_conclusao.append(elemento)
        except Exception as ex:
            print(f"Erro ao clonar o repo: {git_url} - {str(ex)}")

    def clona_repositorios(self):
        try:
          for i in range(self.qtd_repositorios):
              nome = get_repository_name(url=self.lista_urls[i])
              tdi = threading.Thread(target=self.my_clone, args=(self.lista_urls[i], nome))
              tdi.name = 'Thread-'+str(i+1)
              inicio_thread = datetime.datetime.now()
              print(f"Thread {i+1} iniciada em {inicio_thread} - clonando {nome}:{self.lista_urls[i]}...")
              self.lista_threads.append(tdi)
              elemento = (nome, inicio_thread)
              self.lista_tempo_inicio.append(elemento)
              tdi.start()

          for each in self.lista_threads:
              each.join()
        except Exception as ex:
          print(f'Erro no clona_repositorio: {str(ex)}')

    def mostra_tempo_clonagens(self):
        for t1 in self.lista_tempo_inicio:
            for t2 in self.lista_tempo_conclusao:
                if t1[0] == t2[0]:
                    print(f"Tempo de clonagem do {t1[0]} é: {t2[1]-t1[1]} s")

class CSVFile:
    def __init__(self, file_path):
        self.file_path = file_path

    def read(self):
        with open(self.file_path, 'r') as csv_file:
            csv_reader = csv.reader(csv_file)
            for row in csv_reader:
                yield row

class CSVCombiner:
    def __init__(self, folder_path):
        self.folder_path = folder_path
        self.accumulated_content = []

    def combine(self):
        for filename in os.listdir(self.folder_path):
            if filename.endswith('.csv'):
                file_path = os.path.join(self.folder_path, filename)
                csv_file = CSVFile(file_path)
                for row in csv_file.read():
                  if 'name' not in row and 'lucene-solr' not in row and 'jackson' not in row:
                      self.accumulated_content.append(row)

    def write(self, output_file_path):
        with open(output_file_path, 'w', newline='') as output_file:
            csv_writer = csv.writer(output_file)
            csv_writer.writerows(self.accumulated_content)

class CSVModifier:
    def __init__(self, csv_file):
        self.csv_file = csv_file

    def insert_new_line(self, new_line):
        existing_content = []
        with open(self.csv_file.file_path, 'r') as csv_file:
            csv_reader = csv.reader(csv_file)
            for row in csv_reader:
                existing_content.append(row)

        existing_content.insert(0, new_line)

        with open(self.csv_file.file_path, 'w', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerows(existing_content)

### List the repositories for analysis

In [36]:
df_repositorios = pd.read_csv('/content/my_repositories.csv')
lista_repositorios = df_repositorios.url.to_list()
print(lista_repositorios)

['https://github.com/apache/accumulo.git', 'https://github.com/apache/activemq.git', 'https://github.com/apache/ant-ivy.git', 'https://github.com/apache/cassandra.git', 'https://github.com/apache/chukwa.git', 'https://github.com/apache/druid.git', 'https://github.com/google/guava.git', 'https://github.com/apache/jackrabbit.git', 'https://github.com/FasterXML/jackson-core', 'https://github.com/apache/jena.git', 'https://github.com/apache/jspwiki.git', 'https://github.com/apache/lucene.git', 'https://github.com/apache/mina.git', 'https://github.com/apache/pdfbox.git', 'https://github.com/pgjdbc/pgjdbc.git', 'https://github.com/apache/poi.git', 'https://github.com/apache/struts.git', 'https://github.com/cbeust/testng.git', 'https://github.com/apache/tika.git', 'https://github.com/apache/xerces2-j.git']


### Clone repositories locally


In [37]:
qtd_repositorios = len(lista_repositorios)
print(f'Clonando os {qtd_repositorios} repositórios. Aguarde...')
cloner = RepoCloner(qtd_repositorios=len(lista_repositorios), lista_urls=lista_repositorios)
cloner.clona_repositorios()
cloner.mostra_tempo_clonagens()
print(f'Clonagem de todos os {qtd_repositorios} concluída com sucesso!')

Clonando os 20 repositórios. Aguarde...
Thread 1 iniciada em 2023-08-19 16:16:48.005907 - clonando accumulo:https://github.com/apache/accumulo.git...
Thread 2 iniciada em 2023-08-19 16:16:48.007693 - clonando activemq:https://github.com/apache/activemq.git...
Thread 3 iniciada em 2023-08-19 16:16:48.008918 - clonando ant-ivy:https://github.com/apache/ant-ivy.git...
Thread 4 iniciada em 2023-08-19 16:16:48.016845 - clonando cassandra:https://github.com/apache/cassandra.git...
Thread 5 iniciada em 2023-08-19 16:16:48.019351 - clonando chukwa:https://github.com/apache/chukwa.git...
Thread 6 iniciada em 2023-08-19 16:16:48.022995 - clonando druid:https://github.com/apache/druid.git...
Thread 7 iniciada em 2023-08-19 16:16:48.028683 - clonando guava:https://github.com/google/guava.git...
Thread 8 iniciada em 2023-08-19 16:16:48.043137 - clonando jackrabbit:https://github.com/apache/jackrabbit.git...
Thread 9 iniciada em 2023-08-19 16:16:48.048123 - clonando jackson-core:https://github.com/F

### Analyzes CLOC and other metrics for each cloned repository

In [38]:
for each in lista_repositorios:
  try:
    URL_REPOSITORY = each
    target_directory = get_repository_name(url=URL_REPOSITORY)
    target_directory = '/content/'+target_directory
    json_file = target_directory + '.json'

    print(f'Analise CLOC diretório: {target_directory}')
    directory_analyzer = DirectoryAnalyzer(target_directory=target_directory, cloc_output_file=json_file)
    total_nFiles, total_unique_files, total_blank_loc, total_comment_loc, total_java_loc, total_files_loc = directory_analyzer.cloc()

    my_dict=gera_my_dict(URL_REPOSITORY,total_nFiles, total_unique_files, total_blank_loc, total_comment_loc, total_java_loc, total_files_loc)

    print(f'Gera o .csv do {URL_REPOSITORY}')
    gera_csv(URL_REPOSITORY,my_dict)
  except Exception as ex:
    print(f'Erro ao fazer a análise CLOC do repositório {each}')

Analise CLOC diretório: /content/accumulo
Gera o .csv do https://github.com/apache/accumulo.git
Analise CLOC diretório: /content/activemq
Gera o .csv do https://github.com/apache/activemq.git
Analise CLOC diretório: /content/ant-ivy
Gera o .csv do https://github.com/apache/ant-ivy.git
Analise CLOC diretório: /content/cassandra
Gera o .csv do https://github.com/apache/cassandra.git
Analise CLOC diretório: /content/chukwa
Gera o .csv do https://github.com/apache/chukwa.git
Analise CLOC diretório: /content/druid
Gera o .csv do https://github.com/apache/druid.git
Analise CLOC diretório: /content/guava
Gera o .csv do https://github.com/google/guava.git
Analise CLOC diretório: /content/jackrabbit
Gera o .csv do https://github.com/apache/jackrabbit.git
Analise CLOC diretório: /content/jackson-core
Gera o .csv do https://github.com/FasterXML/jackson-core
Analise CLOC diretório: /content/jena
Gera o .csv do https://github.com/apache/jena.git
Analise CLOC diretório: /content/jspwiki
Gera o .csv 

In [39]:
!rm -rf /content/temp  && mkdir /content/temp
!cp /content/*.csv /content/temp
!rm /content/temp/my_repositories.csv

In [40]:
!ls -liath /content/temp

total 88K
 303010 drwxr-xr-x 2 root root 4.0K Aug 19 16:29 .
 303031 -rw-r--r-- 1 root root  216 Aug 19 16:29 xerces2-j.csv
 303025 -rw-r--r-- 1 root root  215 Aug 19 16:29 pdfbox.csv
 303026 -rw-r--r-- 1 root root  212 Aug 19 16:29 pgjdbc.csv
 303027 -rw-r--r-- 1 root root  213 Aug 19 16:29 poi.csv
 303028 -rw-r--r-- 1 root root  216 Aug 19 16:29 struts.csv
 303029 -rw-r--r-- 1 root root  213 Aug 19 16:29 testng.csv
 303030 -rw-r--r-- 1 root root  212 Aug 19 16:29 tika.csv
 303011 -rw-r--r-- 1 root root  217 Aug 19 16:29 accumulo.csv
 303012 -rw-r--r-- 1 root root  219 Aug 19 16:29 activemq.csv
 303013 -rw-r--r-- 1 root root  213 Aug 19 16:29 ant-ivy.csv
 303014 -rw-r--r-- 1 root root  222 Aug 19 16:29 cassandra.csv
 303015 -rw-r--r-- 1 root root  209 Aug 19 16:29 chukwa.csv
 303016 -rw-r--r-- 1 root root  220 Aug 19 16:29 druid.csv
 303017 -rw-r--r-- 1 root root  215 Aug 19 16:29 guava.csv
 303018 -rw-r--r-- 1 root root  220 Aug 19 16:29 jackrabbit.csv
 303019 -rw-r--r-- 1 root root 

### Merge all .csv

In [41]:
folder_path = '/content/temp'
output_file_path = '/content/temp/combined.csv'
csv_combiner = CSVCombiner(folder_path)
csv_combiner.combine()
csv_combiner.write(output_file_path)

csv_file_path = '/content/temp/combined.csv'
new_line = ['name','files','java','blank_loc_java','comment_loc_java','loc_java','loc_files','commit_inicial','commit_final','qtd_commits','qtd_releases']
csv_file = CSVFile(csv_file_path)
csv_modifier = CSVModifier(csv_file)
csv_modifier.insert_new_line(new_line)

### Generates the Dataframe of the CLOC analyzes of the repositories

In [42]:
df_cloc_repositories = pd.read_csv('/content/temp/combined.csv')
df_cloc_repositories

,name,files,java,blank_loc_java,comment_loc_java,loc_java,loc_files,commit_inicial,commit_final,qtd_commits,qtd_releases
0,ant-ivy,1868,664,16475,25102,77401,316140,2005-06-16,2023-08-17,3045,22
1,struts,3346,2542,51338,111898,208008,267837,2006-02-22,2023-08-19,6705,143
2,jackson-core,372,338,9881,23168,76265,78170,2011-12-22,2023-08-18,2600,140
3,jspwiki,1266,749,16696,38059,58559,119588,2007-10-06,2023-08-07,9309,111
4,tika,2221,1679,31987,64616,166064,220666,2007-03-31,2023-08-18,7613,95
5,mina,348,319,6044,13734,23633,25305,2005-12-28,2023-04-30,2409,72
6,guava,1968,1909,59867,128589,356714,361054,2009-06-18,2023-08-14,6155,101
7,pgjdbc,828,544,15100,20178,81127,110057,1997-08-16,2023-08-03,3315,211
8,xerces2-j,1134,833,32128,87734,140644,181208,1999-11-09,2019-04-06,5510,98
9,testng,2434,2201,20000,10799,99362,106252,2006-07-30,2023-08-17,5275,95
